In [2]:
import streamlit as st
from streamlit_chat import message
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.chat_models import ChatOpenAI
from langchain.chains import ConversationalRetrievalChain
from langchain.document_loaders.csv_loader import CSVLoader
from langchain.vectorstores import FAISS
import tempfile
import json

In [5]:
f = open("apikey.json")
data = json.load(f)
key = data["openaikey"]

In [7]:
user_api_key = st.sidebar.text_input(
    label="#### Your OpenAI API key 👇",
    placeholder=key,
    type="password")

In [8]:
loader = CSVLoader(file_path="./RAW_recipes.csv", encoding="utf-8", csv_args={
            'delimiter': ','})
data = loader.load()
st.write(data)

In [10]:
embeddings = OpenAIEmbeddings(openai_api_key=key)
vectorstore = FAISS.from_documents(data, embeddings)

Retrying langchain.embeddings.openai.embed_with_retry.<locals>._embed_with_retry in 4.0 seconds as it raised RateLimitError: You exceeded your current quota, please check your plan and billing details..
Retrying langchain.embeddings.openai.embed_with_retry.<locals>._embed_with_retry in 4.0 seconds as it raised RateLimitError: You exceeded your current quota, please check your plan and billing details..
Retrying langchain.embeddings.openai.embed_with_retry.<locals>._embed_with_retry in 4.0 seconds as it raised RateLimitError: You exceeded your current quota, please check your plan and billing details..
Retrying langchain.embeddings.openai.embed_with_retry.<locals>._embed_with_retry in 8.0 seconds as it raised RateLimitError: You exceeded your current quota, please check your plan and billing details..
Retrying langchain.embeddings.openai.embed_with_retry.<locals>._embed_with_retry in 10.0 seconds as it raised RateLimitError: You exceeded your current quota, please check your plan and bi

RateLimitError: You exceeded your current quota, please check your plan and billing details.